In [1]:
import torch
from helper.model import *
from helper.utils import *
from helper.distance_map import *
from helper.evaluate import *
import pandas as pd
import warnings
def warn(*args, **kwargs):
    pass
warnings.warn = warn

In [2]:
# args_train_data = "uniref1/uniref1_train_split_0"
# args_test_data = "uniref1/uniref1_test_split_0_curate"
# args_model_name = "uniref1/uniref1_split0_triplet"
 
args_train_data = "uniref100_full"
args_test_data = "new"
args_model_name = "uniref100_full4" 
 
use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")
dtype = torch.float32
# load id ec from tran and test
id_ec_train, ec_id_dict_train = get_ec_id_dict(
    './data/' + args_train_data + '.csv')
id_ec_test, _ = get_ec_id_dict(
    './data/' + args_test_data + '.csv')

# load model
if False:
    # no model used for pretrained embedding
    model = lambda *args: args[0]
else:
    model = LayerNormNet(512, 128, device, dtype)
    checkpoint = torch.load('./model/' + args_model_name + '.pth')
    model.load_state_dict(checkpoint)
    model.eval()

# compute distance map
emb_train = model(esm_embedding(ec_id_dict_train, device, dtype))
emb_test = model_embedding_test(id_ec_test, model, device, dtype)

100%|██████████| 5242/5242 [01:12<00:00, 72.18it/s] 


In [3]:
eval_dist = get_dist_map_test(
        emb_train, emb_test, ec_id_dict_train, id_ec_test, 
        device, dtype, dot=False)
eval_df = pd.DataFrame.from_dict(eval_dist)

The embedding sizes for train and test: torch.Size([241025, 128]) torch.Size([392, 128])


100%|██████████| 5242/5242 [00:00<00:00, 14879.30it/s]


Calculating eval distance map, between 392 test ids and 5242 train EC cluster centers


392it [00:00, 1076.00it/s]


In [7]:
def get_pred_probs(out_filename, pred_type="_maxsep"):
    file_name = out_filename+pred_type
    result = open(file_name+'.csv', 'r')
    csvreader = csv.reader(result, delimiter=',')
    pred_probs = []
    for row in csvreader:
        preds_ec_lst = []
        preds_with_dist = row[1:]
        probs = torch.zeros(len(preds_with_dist))
        count = 0
        
        for pred_ec_dist in preds_with_dist:
            # get EC number 3.5.2.6 from EC:3.5.2.6/10.8359
            ec_i = - float(pred_ec_dist.split(":")[1].split("/")[1])
            probs[count] = ec_i
            #preds_ec_lst.append(probs)
            count += 1
        # sigmoid of the negative distances 
        # probs = (1 - torch.exp(-1/probs)) / (1 + torch.exp(-1/probs))
        # probs = probs/torch.sum(probs)
        probs = torch.nn.functional.softmax(probs)
        pred_probs.append(probs)
        
    return pred_probs


def get_pred_dist(out_filename, pred_type="_maxsep"):
    file_name = out_filename+pred_type
    result = open(file_name+'.csv', 'r')
    csvreader = csv.reader(result, delimiter=',')
    pred_probs = []
    for row in csvreader:
        preds_ec_lst = []
        preds_with_dist = row[1:]
        probs = torch.zeros(len(preds_with_dist))
        count = 0
        
        for pred_ec_dist in preds_with_dist:
            # get EC number 3.5.2.6 from EC:3.5.2.6/10.8359
            ec_i = float(pred_ec_dist.split(":")[1].split("/")[1])
            probs[count] = ec_i
            #preds_ec_lst.append(probs)
            count += 1
        # sigmoid of the negative distances 
       
        
        pred_probs.append(probs)
    return pred_probs

In [5]:
pred_dist = get_pred_dist(out_filename, pred_type='_maxsep')
 

NameError: name 'out_filename' is not defined

In [ ]:
torch.nn.functional.softmax(-pred_dist[3]), pred_probs[3]

(tensor([0.5083, 0.4917]), tensor([0.4939, 0.5061]))

In [ ]:
for label, prob, dist in zip(pred_label, pred_probs, pred_dist):
    print(list(label), list(prob.numpy()), list(dist.numpy()))

['3.1.3.43'] [1.0] [-6.5798883]
['1.14.13.7'] [1.0] [-0.7705774]
['2.7.7.15'] [1.0] [-4.5944805]
['4.1.3.27', '3.5.1.129'] [0.50830215, 0.49169782] [-1.2449332, -1.2117215]
['3.1.26.11'] [1.0] [-3.530164]
['2.3.1.158'] [1.0] [-3.856379]
['1.14.19.9'] [1.0] [-0.4354036]
['2.7.1.31', '5.6.1.7', '1.1.1.42'] [0.35090867, 0.34047744, 0.30861387] [-0.7173605, -0.6871834, -0.5889254]
['3.2.2.31'] [1.0] [-2.6620586]
['1.15.1.1'] [1.0] [-1.849775]
['3.2.1.171'] [1.0] [-4.164794]
['3.2.1.15'] [1.0] [-1.6179928]
['1.10.3.2'] [1.0] [-1.8364525]
['1.14.13.9'] [1.0] [-0.9927875]
['1.1.1.149'] [1.0] [-0.7768541]
['1.11.1.7'] [1.0] [-3.4823935]
['3.1.1.74'] [1.0] [-2.4632025]
['3.5.2.6'] [1.0] [-0.5758673]
['1.11.1.7'] [1.0] [-1.7261755]
['2.2.1.2'] [1.0] [-2.7199898]
['2.3.1.12', '3.2.1.118'] [0.52517736, 0.47482258] [-2.1159656, -2.0151708]
['2.7.7.14'] [1.0] [-3.7933033]
['2.7.7.15'] [1.0] [-5.464615]
['5.4.2.2'] [1.0] [-4.0838566]
['3.4.19.3'] [1.0] [-6.1451087]
['3.1.4.53', '3.1.4.17'] [0.5132673

In [5]:
out_filename = './eval/' + args_test_data
write_max_sep_choices(eval_df, out_filename, first_grad=True, use_max_grad=False)
pred_label = get_pred_labels(out_filename, pred_type='_maxsep')
pred_probs= get_pred_probs(out_filename, pred_type='_maxsep')

In [ ]:
print(true_label[3])
true_m[3] = mlb.transform([true_label[3]])
mlb.transform([true_label[3]]).sum()

['4.3.3.6', '3.5.1.2']


2

In [ ]:
len(true_label)

1313

In [8]:
def get_ec_pos_dict(mlb, true_label, pred_label):
    ec_list = []
    pos_list = []
    for i in range(len(true_label)):
        ec_list += list(mlb.inverse_transform(mlb.transform([true_label[i]]))[0])
        pos_list += list(np.nonzero(mlb.transform([true_label[i]]))[1])
    for i in range(len(pred_label)):
        ec_list += list(mlb.inverse_transform(mlb.transform([pred_label[i]]))[0])
        pos_list += list(np.nonzero(mlb.transform([pred_label[i]]))[1])
    label_pos_dict = {}
    for i in range(len(ec_list)):
        ec, pos = ec_list[i], pos_list[i]
        label_pos_dict[ec] = pos
        
    return label_pos_dict

In [ ]:
label_pos_dict[labels[0]]

382

In [ ]:
label_pos_dict = get_ec_pos_dict(mlb, true_label, pred_label)
pred_m = np.zeros((n_test, len(mlb.classes_)))
for i in range(n_test):
    labels, probs = pred_label[i], pred_probs[i]
    for label, prob in zip(labels, probs):
        if label in all_label:
            pos = label_pos_dict[label]
            pred_m[i] = prob


In [ ]:
roc

0

In [9]:
mlb = MultiLabelBinarizer()
mlb.fit([list(all_label)])
n_test = len(pred_label)
pred_m = np.zeros((n_test, len(mlb.classes_)))
true_m = np.zeros((n_test, len(mlb.classes_)))
pred_m_auc = np.zeros((n_test, len(mlb.classes_)))
for i in range(n_test):
    pred_m[i] = mlb.transform([pred_label[i]])
    true_m[i] = mlb.transform([true_label[i]])
    # fill in probabilities for prediction
    labels, probs = pred_label[i], pred_probs[i]
    for label, prob in zip(labels, probs):
        if label in all_label:
            pos = label_pos_dict[label]
            pred_m_auc[i, pos] = prob

    
pre = precision_score(true_m, pred_m, average='weighted', zero_division=0)
rec = recall_score(true_m, pred_m, average='weighted')
f1 = f1_score(true_m, pred_m, average='weighted')
roc = roc_auc_score(true_m, pred_m_auc, average='weighted')
acc = accuracy_score(true_m, pred_m)

NameError: name 'all_label' is not defined

In [ ]:
pred_probs

[tensor([1.]),
 tensor([1.]),
 tensor([1.]),
 tensor([0.5083, 0.4917]),
 tensor([1.]),
 tensor([1.]),
 tensor([1.]),
 tensor([0.3509, 0.3405, 0.3086]),
 tensor([1.]),
 tensor([1.]),
 tensor([1.]),
 tensor([1.]),
 tensor([1.]),
 tensor([1.]),
 tensor([1.]),
 tensor([1.]),
 tensor([1.]),
 tensor([1.]),
 tensor([1.]),
 tensor([1.]),
 tensor([0.5252, 0.4748]),
 tensor([1.]),
 tensor([1.]),
 tensor([1.]),
 tensor([1.]),
 tensor([0.5133, 0.4867]),
 tensor([1.]),
 tensor([1.]),
 tensor([1.]),
 tensor([1.]),
 tensor([1.]),
 tensor([1.]),
 tensor([0.5406, 0.4594]),
 tensor([1.]),
 tensor([1.]),
 tensor([0.5300, 0.4700]),
 tensor([1.]),
 tensor([1.]),
 tensor([1.]),
 tensor([1.]),
 tensor([1.]),
 tensor([1.]),
 tensor([1.]),
 tensor([0.6208, 0.3792]),
 tensor([1.]),
 tensor([1.]),
 tensor([0.5221, 0.4779]),
 tensor([1.]),
 tensor([1.]),
 tensor([1.]),
 tensor([0.5000, 0.5000]),
 tensor([1.]),
 tensor([1.]),
 tensor([1.]),
 tensor([1.]),
 tensor([1.]),
 tensor([1.]),
 tensor([1.]),
 tensor([0.378

In [ ]:
def write_max_sep_choices_new(df, csv_name, first_grad=True, use_max_grad=False):
    out_file = open(csv_name + '_maxsep.csv', 'w', newline='')
    csvwriter = csv.writer(out_file, delimiter=',')
    all_test_EC = set()
    for col in df.columns:
        ec = []
        smallest_10_dist_df = df[col].nsmallest(10)
        dist_lst = list(smallest_10_dist_df)
        max_sep_i = maximum_separation(dist_lst, first_grad, use_max_grad)

        background_noise = 0
        for i in range(max_sep_i+1, 10):
            dist_i = smallest_10_dist_df[i]
            background_noise += dist_i
        background_noise = background_noise/len(range(max_sep_i+1, 10))
        
        for i in range(max_sep_i+1):
            EC_i = smallest_10_dist_df.index[i]
            dist_i = smallest_10_dist_df[i] - background_noise
            dist_str = "{:.7f}".format(dist_i)
            all_test_EC.add(EC_i)
            ec.append('EC:' + str(EC_i) + '/' + dist_str)
        
        

        
        ec.insert(0, col)
        csvwriter.writerow(ec)
    return

In [ ]:
mlb = MultiLabelBinarizer()
mlb.fit([list(all_label)])
n_test = len(pred_label)
pred_m = np.zeros((n_test, len(mlb.classes_)))
true_m = np.zeros((n_test, len(mlb.classes_)))
# for including probability
pred_m_auc = np.zeros((n_test, len(mlb.classes_)))
label_pos_dict = get_ec_pos_dict(mlb, true_label, pred_label)
for i in range(n_test):
    pred_m[i] = mlb.transform([pred_label[i]])
    true_m[i] = mlb.transform([true_label[i]])
        # fill in probabilities for prediction
    labels, probs = pred_label[i], pred_probs[i]
    for label, prob in zip(labels, probs):
        if label in all_label:
            pos = label_pos_dict[label]
            pred_m_auc[i, pos] = prob
            print(prob)
 
roc = roc_auc_score(true_m, pred_m_auc, average='weighted')
 
roc

tensor(1.)
tensor(1.)
tensor(1.)
tensor(0.5083)
tensor(1.)
tensor(0.3405)
tensor(0.3086)
tensor(1.)
tensor(1.)
tensor(1.)
tensor(1.)
tensor(1.)
tensor(1.)
tensor(1.)
tensor(1.)
tensor(1.)
tensor(1.)
tensor(1.)
tensor(1.)
tensor(1.)
tensor(0.5133)
tensor(0.4867)
tensor(1.)
tensor(1.)
tensor(1.)
tensor(1.)
tensor(1.)
tensor(1.)
tensor(0.4594)
tensor(1.)
tensor(1.)
tensor(0.4700)
tensor(1.)
tensor(1.)
tensor(1.)
tensor(1.)
tensor(1.)
tensor(1.)
tensor(0.6208)
tensor(0.3792)
tensor(1.)
tensor(1.)
tensor(0.5221)
tensor(1.)
tensor(1.)
tensor(1.)
tensor(1.)
tensor(1.)
tensor(1.)
tensor(1.)
tensor(1.)
tensor(1.)
tensor(0.5661)
tensor(0.4339)
tensor(1.)
tensor(1.)
tensor(1.)
tensor(1.)
tensor(1.)
tensor(1.)
tensor(1.)
tensor(1.)
tensor(1.)
tensor(0.5339)
tensor(0.4661)
tensor(1.)
tensor(0.5635)
tensor(0.4365)
tensor(1.)
tensor(1.)
tensor(1.)
tensor(1.)
tensor(1.)
tensor(1.)
tensor(0.5349)
tensor(0.4651)
tensor(1.)
tensor(0.5127)
tensor(1.)
tensor(1.)
tensor(1.)
tensor(1.)
tensor(1.)
tensor(1.)


0.8201263661816239

In [11]:
eval_df = pd.DataFrame.from_dict(eval_dist)
# write the top 10 closest EC to _top10.csv
out_filename = './eval/' + args_test_data
# _ = write_top10_choices(eval_df, out_filename)
# maximum separation results
write_max_sep_choices_new(eval_df, out_filename,
                        first_grad=True,
                        use_max_grad=False)
# get preds and true labels
pred_label = get_pred_labels(out_filename, pred_type='_maxsep')
pred_probs = get_pred_probs(out_filename, pred_type='_maxsep')
true_label, all_label = get_true_labels('./data/'+args_test_data)
pre, rec, f1, roc, acc = get_eval_metrics(
    pred_label, pred_probs, true_label, all_label)
print("############ EC calling results using maximum separation ############")
print('-' * 75)
print(f'>>> total samples: {len(true_label)} | total ec {len(all_label)} |\n'
        f'precision | recall | F1 | AUC | accuracy' )
print( f'{pre:.5} , {rec:.5} , {f1:.5} , {roc:.7} , {acc:.5}')
print('-' * 75)

############ EC calling results using maximum separation ############
---------------------------------------------------------------------------
>>> total samples: 392 | total ec 177 |
precision | recall | F1 | AUC | accuracy
0.62227 , 0.49901 , 0.51772 , 0.7490581 , 0.51276
---------------------------------------------------------------------------


python -u .\eval_maxsep_BLAST.py -tr uniref100/uniref100_train_split_0 -te uniref100/uniref100_test_split_0_curate -n uniref100/uniref100_split_0_triplet >> weighted.out; python -u .\eval_maxsep_BLAST.py -tr uniref100/uniref100_train_split_1 -te uniref100/uniref100_test_split_1_curate -n uniref100/uniref100_split_1_triplet >> weighted.out; python -u .\eval_maxsep_BLAST.py -tr uniref100/uniref100_train_split_2 -te uniref100/uniref100_test_split_2_curate -n uniref100/uniref100_split_2_triplet >> weighted.out; python -u .\eval_maxsep_BLAST.py -tr uniref100/uniref100_train_split_3 -te uniref100/uniref100_test_split_3_curate -n uniref100/uniref100_split_3_triplet >> weighted.out; python -u .\eval_maxsep_BLAST.py -tr uniref100/uniref100_train_split_4 -te uniref100/uniref100_test_split_4_curate -n uniref100/uniref100_split_4_triplet >> weighted.out

In [ ]:
def write_max_sep_choices_new(df, csv_name, first_grad=True, use_max_grad=False):
    out_file = open(csv_name + '_maxsep.csv', 'w', newline='')
    csvwriter = csv.writer(out_file, delimiter=',')
    all_test_EC = set()
    for col in df.columns:
        ec = []
        smallest_10_dist_df = df[col].nsmallest(10)
        dist_lst = list(smallest_10_dist_df)
        max_sep_i = maximum_separation(dist_lst, first_grad, use_max_grad)

        background_noise = 0
        for i in range(max_sep_i+1, 10):
            dist_i = smallest_10_dist_df[i]
            background_noise += dist_i
        background_noise = background_noise/len(range(max_sep_i+1, 10))
        
        for i in range(max_sep_i+1):
            EC_i = smallest_10_dist_df.index[i]
            dist_i = smallest_10_dist_df[i] - background_noise
            dist_str = "{:.7f}".format(dist_i)
            all_test_EC.add(EC_i)
            ec.append('EC:' + str(EC_i) + '/' + dist_str)

        ec.insert(0, col)
        csvwriter.writerow(ec)
    return